# Исследование надежности заемщиков.

# Шаг 1. Обзор данных

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим отсутствующие значения в количественных колонках `days_employed` (трудовой стаж), `total_income` (ежемесячный доход). <p>Обратим внимание, что количество значений в обеих колонках - одинаковое, по 19 351, из чего сделаем вывод, что пропуски носят не слуайный характер, а скорее всего связаны с наличием выданных кредитов людям без постоянного места работы.</p> <p>Количество пропусков составляет **10%** от общего количества записей. Учитывая, что цель исследования - оценить влияение семейного положения на платежеспособность клиента, заполним пропущенные значения медианными. <p>Также, изучим подробнее данные в других колонках таблицы.

In [6]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим два значения колонки `children`, с аномальными значениями: -1 и 20. Суммарно, это 123 значения или 0,5% от общего количества данных. Изменим -1 на 1. Записи со значением 20 удалим из таблицы.  

In [7]:
data['children'] = abs(data['children']) #присваиваем столбцу значения по модулю, таким образом значения -1 преобразуются в 1
data = data.loc[data['children']!=20].reset_index(drop=True) #избавляемся от значений =20
data['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Видно, что часть значений колонки `days_employed` отрицательные. Изучим ее подробнее.  

In [8]:
positive_stage_count = data['days_employed'][data['days_employed']>=0].count()
negative_stage_count = data['days_employed'][data['days_employed']<0].count()
positive_stage_mean = data['days_employed'][data['days_employed']>=0].mean()
positive_stage_min = data['days_employed'][data['days_employed']>=0].min()
positive_stage_mean_age = data['dob_years'][data['days_employed']>=0].median()
negative_stage_mean = data['days_employed'][data['days_employed']<0].mean()


print ('Количество положительных значений стажа:',positive_stage_count)
print ('Количество отрицательных значений стажа:',negative_stage_count)
print ('Среднее положительное значение стажа:', int(positive_stage_mean),'дней', 'что составляет', int(positive_stage_mean/365), 'лет')
print ('Минимальное положительное значение стажа состовляет:', int(positive_stage_min),'дней')
print ('Медианный возраст заемщиков с положительным значением стажа:', int(positive_stage_mean_age))
print ('Среднее отрицательное значение стажа:', int(negative_stage_mean),'дней', 'что составляет', int(negative_stage_mean/365), 'лет')

Количество положительных значений стажа: 3438
Количество отрицательных значений стажа: 15846
Среднее положительное значение стажа: 365012 дней что составляет 1000 лет
Минимальное положительное значение стажа состовляет: 328728 дней
Медианный возраст заемщиков с положительным значением стажа: 60
Среднее отрицательное значение стажа: -2353 дней что составляет -6 лет


**Выводы**

1. Все положительные значения в колонке `days_employed` не могут указывать на количество дней трудового стажа, так как даже минимальное положительное значение значительно превышает средний возраст заемщика. Предположительно, значения указаны в часах. 
2. Отрицательные значения в колонке `days_employed`, исходя из проведенного анализа, показывают количество дней трудового стажа. То, что они отрицаительные - случайная ошибка, которую необходимо исправить. 

In [9]:
def hours_to_years (days):
    if days>0:
        days = days/24/365
    return days

data['days_employed'] = data['days_employed'].apply(hours_to_years) #переводим положительныез значения в дни
data['days_employed'] = abs(data['days_employed']) #избавляемся от отрицательных значений

In [10]:
print(data['dob_years'].max())
print (data['dob_years'].min())
(data[data['dob_years']==0])

75
0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,39.559546,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,45.417416,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1038,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19760,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20391,0,38.668364,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20506,0,37.870008,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21107,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Учитывая, что целью исследования стоит выяснение, влияет ли семейное положение на платежеспособность, заполним нулевые значения возраста медианным значением. 

In [71]:
def median_age (age):
    new_age = int(data['dob_years'].median())
    if age==0:
        age = new_age
    return age

data['dob_years'] = data['dob_years'].apply(median_age)
print ('Минимальный возраст заемщика:',data['dob_years'].min(), 'лет')

Минимальный возраст заемщика: 19 лет


In [12]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15173
высшее                  5246
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

In [13]:
data['education_id'].value_counts()

1    15173
0     5246
2      742
3      282
4        6
Name: education_id, dtype: int64

Сведения о высшем образовании в таблице выглядят корректными. 

In [72]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12290
гражданский брак          4138
не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

Сведения о семейном статусе выглядят корректными. 

In [15]:
data['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

Данные о половой принадлежности выглядят корректно, за исключением одного значения. Учитывая цели исследования оставим данное значение без изменений. 

In [16]:
data['income_type'].value_counts()

сотрудник          11076
компаньон           5063
пенсионер           3847
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Данные о способе получения дохода выглядят корректно. 

In [17]:
data['debt'].value_counts()

0    19716
1     1733
Name: debt, dtype: int64

Данные о наличии/отсутствии задолженности выглядят корректно.

In [74]:
print ('Максимальное значение дохода составляет', data['total_income'].max())
print ('Минимальное значение дохода составляет', data['total_income'].min())


Максимальное значение дохода составляет 2265604
Минимальное значение дохода составляет 20667


Данные о суммах дохода выглядят корректно.

In [19]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     773
сыграть свадьбу                           770
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   652
операции с жильем                         649
операции с коммерческой недвижимостью     647
покупка жилья                             644
жилье                                     643
покупка жилья для семьи                   640
недвижимость                              633
строительство собственной недвижимости    629
операции со своей недвижимостью           629
строительство жилой недвижимости          625
покупка недвижимости                      622
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              610
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

Необходимо выполнить категоризацию значений по целям кредита. 

# Шаг 2 Заполнение пропусков

In [20]:
median_stage = data['days_employed'].median()
data['days_employed']=data['days_employed'].fillna(median_stage)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21449 entries, 0 to 21448
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21449 non-null  int64  
 1   days_employed     21449 non-null  float64
 2   dob_years         21449 non-null  int64  
 3   education         21449 non-null  object 
 4   education_id      21449 non-null  int64  
 5   family_status     21449 non-null  object 
 6   family_status_id  21449 non-null  int64  
 7   gender            21449 non-null  object 
 8   income_type       21449 non-null  object 
 9   debt              21449 non-null  int64  
 10  total_income      19284 non-null  float64
 11  purpose           21449 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [21]:
median_total_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_total_income)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21449 entries, 0 to 21448
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21449 non-null  int64  
 1   days_employed     21449 non-null  float64
 2   dob_years         21449 non-null  int64  
 3   education         21449 non-null  object 
 4   education_id      21449 non-null  int64  
 5   family_status     21449 non-null  object 
 6   family_status_id  21449 non-null  int64  
 7   gender            21449 non-null  object 
 8   income_type       21449 non-null  object 
 9   debt              21449 non-null  int64  
 10  total_income      21449 non-null  float64
 11  purpose           21449 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Шаг 3. Изменение типов данных.

Изменим типы данных с вещественного на целочисленные в столбцах: `days_employed`, `total_income`

In [22]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21449 entries, 0 to 21448
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21449 non-null  int64 
 1   days_employed     21449 non-null  int64 
 2   dob_years         21449 non-null  int64 
 3   education         21449 non-null  object
 4   education_id      21449 non-null  int64 
 5   family_status     21449 non-null  object
 6   family_status_id  21449 non-null  int64 
 7   gender            21449 non-null  object
 8   income_type       21449 non-null  object
 9   debt              21449 non-null  int64 
 10  total_income      21449 non-null  int64 
 11  purpose           21449 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


# Шаг 4. Удаление дубликатов.

In [23]:
print ('количетсво явных дубликатов в строках таблицы:',data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True) #удаляем явные дубликаты
print ('количетсво явных дубликатов в строках таблицы после удаления:',data.duplicated().sum())

количетсво явных дубликатов в строках таблицы: 72
количетсво явных дубликатов в строках таблицы после удаления: 0


# Шаг 5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [24]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


В целях упрощения датафрейма, создадим отдельные словари для образования - `education_dict` и семейного статуса `family_status_dict`. 

In [25]:
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

credit_history = data [[ 'children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
credit_history.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


# Шаг 6. Категоризация данных.

In [26]:
#Категоризируем данные о способе получения дохода.
def income_categories (income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

credit_history['total_income_category'] = credit_history['total_income'].apply(income_categories)

/tmp/ipykernel_32/3846242974.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_history['total_income_category'] = credit_history['total_income'].apply(income_categories)


In [27]:
#Категоризируем данные по возрасту клиента.
def age_categories(age):
    if age <= 25:
        return 'junior'
    if age <= 35:
        return 'middle'
    if age <= 50:
        return 'adult'
    return 'retiree'

credit_history['age_group'] = credit_history['dob_years'].apply(age_categories)
credit_history['age_group'].value_counts()

adult      8191
retiree    6626
middle     5332
junior     1228
Name: age_group, dtype: int64

In [28]:
credit_history['total_income_category'].value_counts()

C    15959
B     5021
D      350
A       25
E       22
Name: total_income_category, dtype: int64

In [29]:
credit_history['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           761
операции с недвижимостью                  674
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          623
покупка своего жилья                      620
покупка недвижимости                      619
строительство недвижимости                619
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

In [30]:
#Категоризируем данные по целям кредитов.
def purpose_categories (purpose):
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'недвижим' in purpose:
        return 'недвижимость'
    if 'авто' in purpose:
        return 'автомобиль'
    if 'жиль' in purpose:
        return 'недвижимость'
    if 'образ' in purpose:
        return 'образование'
    return 'остальное'

credit_history ['purpose_category'] = credit_history['purpose'].apply(purpose_categories)

In [31]:
credit_history['purpose_category'].value_counts()

недвижимость    10775
автомобиль       4290
образование      3998
свадьба          2314
Name: purpose_category, dtype: int64

Данные категоризированы. 

# Шаг 7. Анализ данных.

In [32]:
#рассчитаем средний процент должников датафрейма. 

mean_debt = credit_history['debt'].mean()
print ('Средний уровень должников составляет:',mean_debt*100,'%')


Средний уровень задолженности составляет: 8.106843804088507 %


In [34]:
#Зависимость от количества детей
table_children = pd.pivot_table(credit_history, index = 'children', columns = 'debt', values = 'purpose', aggfunc = 'count')
table_children.loc[5,1] = 0
table_children['credits_count'] = table_children[0] + table_children[1]
table_children['credits_count'] = table_children[1]/table_children['credits_count']*100
table_children [['credits_count','credits_count']]


debt,total_borrowers,debitors_percentage
children,,
0,14090.0,7.544358
1,4855.0,9.165808
2,2052.0,9.454191
3,330.0,8.181818
4,41.0,9.756098
5,9.0,0.000000


Отметим негативный тренд выплчиваемости кредита по мере увеличения количества детей. 
Тренд прерывается на значении `3`, но стоит принимать во внимание сравнительно небольшое количество исследований. 

Появление детей в семье может негативно сказаться на возможности выплатить кредит в срок по следующим причинам:  
1. Уменьшение финансового потока, так как один из родителей выходит в декрет.
2. Увеличение суммы обязательных ежемесячных трат.

In [35]:
#Зависимость от семейного положения
marriage = pd.pivot_table(credit_history, index = ['family_status_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')

marriage['total_borrowers'] = marriage[0] + marriage[1]
marriage['debitors_percentage'] = marriage[1]/marriage['total_borrowers']*100


marriage = marriage.merge(family_status_dict, on = 'family_status_id', how = 'left')
marriage[['family_status', 'total_borrowers', 'debitors_percentage']]



,family_status,total_borrowers,debitors_percentage
0,женат / замужем,12290,7.550854
1,гражданский брак,4138,9.304012
2,вдовец / вдова,955,6.596859
3,в разводе,1193,7.041073
4,не женат / не замужем,2801,9.746519


Отметим, что в категориях, где отношения были законодательно оформлены, `женат / замужем`,`вдовец / вдова`,`в разводе`, процент выплачиваемости кредитов значительно лучше, чем в остальных категориях: `не женат / не замужем`,`гражданский брак`

Для исследования причин, проведем дополнительный анализ: 
1. Оценим влияние других категориальных значений фрейма на выплачиваемость кредита. 
2. Проверим, каким образом распределены наиболее влияющие категориальные значения по различным видам семейного положения. 

In [33]:
#Зависимость от суммы дохода
table = pd.pivot_table(credit_history,index = 'total_income_category', columns = ['debt'], values = 'purpose', aggfunc = 'count')
table['credits_count'] = table[0] + table[1]
table['debt_count'] = table[1]/table['credits_count']*100
table 


debt,0,1,credits_count,debt_count
total_income_category,,,,
A,23,2,25,8.000000
B,4667,354,5021,7.050388
C,14605,1354,15959,8.484241
D,329,21,350,6.000000
E,20,2,22,9.090909


Из таблицы выше видно отсутсвие корреляции между уровнем дохода и выплачиваемостью кредита. 
Можно отметить незначительное улучшение платежеспосбности для категории `B`. 
Значения категорий `D` и `E` нельзя считать репрезентативными, так как слишком мало исследований.

In [36]:
#Зависимость от цели кредита
purpose = pd.pivot_table(credit_history, index = 'purpose_category', columns = 'debt', values = 'purpose', aggfunc='count')

purpose['total_borrowers'] = purpose[0] + purpose[1]
purpose['debitors_percentage'] = purpose[1]/purpose['total_borrowers']*100
purpose [['total_borrowers', 'debitors_percentage']]



debt,total_borrowers,debitors_percentage
purpose_category,,
автомобиль,4290,9.347319
недвижимость,10775,7.238979
образование,3998,9.229615
свадьба,2314,7.908384


Из таблицы видно, что выплачиваемость кредита по категориям `свадьба` и `недвижимость` лучше, чем по категориям `автомобиль` и `образование`.

In [47]:
#Возвращаемость кредита в зависимости от пола заемщика. 

gender = pd.pivot_table(credit_history, index ='gender', columns ='debt', values ='purpose_category', aggfunc ='count')
gender['total_borrowers'] = gender[0] + gender[1]
gender['debitors_percentage'] = gender[1]/gender['total_borrowers']*100
gender [['total_borrowers', 'debitors_percentage']]
gender = gender.dropna()
gender[['total_borrowers', 'debitors_percentage']]


debt,total_borrowers,debitors_percentage
gender,,
F,14126.0,7.008353
M,7250.0,10.248276


Несмотря на сомнительную этичность данного разреза, сделаем вывод, что женщины выплачивают кредиты значительно исправнее мужчин. 

In [49]:
#Возвращаемость кредита в зависимости от уровня образования. 

education = pd.pivot_table(credit_history, index = 'education_id', columns = 'debt', values = 'purpose', aggfunc = 'count')
education = education.merge(education_dict, on='education_id', how = 'left')
education['total_borrowers'] = education[0] + education[1]
education['debitors_percentage'] = education[1]/education['total_borrowers']*100

education = education.dropna()
education[['education', 'total_borrowers', 'debitors_percentage']]



,education,total_borrowers,debitors_percentage
0,высшее,5236.0,5.309396
1,среднее,15111.0,8.973595
2,неоконченное высшее,742.0,9.164420
3,начальное,282.0,10.992908


Отметим, что клиенты с высшим образованием возвращают кредиты значительно лучше, клиентов с остальными категориями образований. 

In [48]:
#Возвращаемость кредита в зависимости от способа получения дохода

income_type = pd.pivot_table(credit_history, index = 'income_type', columns = 'debt', values = 'purpose', aggfunc = 'count')
income_type['total_borrowers'] = income_type[0] + income_type[1]
income_type['debitors_percentage'] = income_type[1]/income_type['total_borrowers']*100

income_type = income_type.dropna()
income_type[['total_borrowers', 'debitors_percentage']]



debt,total_borrowers,debitors_percentage
income_type,,
безработный,2.0,50.000000
госслужащий,1455.0,5.910653
компаньон,5056.0,7.397152
пенсионер,3820.0,5.654450
сотрудник,11040.0,9.556159


Отметим, что госслужащие, компаньоны и пенсионеры возвращают кредиты лучше, чем сотрудники. 

In [52]:
age = pd.pivot_table(credit_history, index = 'age_group', columns = 'debt', values = 'purpose', aggfunc = 'count')
age['total_borrowers'] = age[0] + age[1]
age['debitors_percentage'] = age[1]/age['total_borrowers']*100

age = age.dropna()
age[['total_borrowers', 'debitors_percentage']]

debt,total_borrowers,debitors_percentage
age_group,,
adult,8191,8.020999
junior,1228,10.667752
middle,5332,10.596399
retiree,6626,5.734983


# Выводы
1. Увеличение количества детей негативно влияет на выплачиваемость кредита. 
2. Законодательно оформленные отношения позитивно влияют на выплачиваемость кредита. 
3. Наиболее значительное влияние на выплачиваемость кредита оказывают критерии: возрастная группа и уровень образования заемщика.

# Гипотезы
1. Среди клиентов банка, находящихся в законодательно оформленных отношениях доминируют люди старших возрастных групп и люди, имеющие высшее образование.
2. Дети появляются в относительно молодых возрастных группах, что может негативно влиять на выплачиваемость кредита.

Проверим гипотезы, составив необходимые сводные таблицы. 

In [41]:
#Зависимость от семейного положения и образования
marriage_and_education = pd.pivot_table(credit_history, index = ['education_id','family_status_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')

marriage_and_education['total_borrowers'] = marriage_and_education[0] + marriage_and_education[1]
marriage_and_education['debitors_percentage'] = marriage_and_education[1]/marriage_and_education['total_borrowers']*100
marriage_and_education = marriage_and_education.dropna()
marriage_and_education = marriage_and_education.reset_index()
marriage_and_education = marriage_and_education.merge(education_dict, on = 'education_id', how = 'left')
marriage_and_education = marriage_and_education.merge(family_status_dict, on = 'family_status_id', how = 'left')
marriage_and_education[['education', 'family_status', 'total_borrowers', 'debitors_percentage']]




,education,family_status,total_borrowers,debitors_percentage
0,высшее,женат / замужем,3030.0,5.115512
1,высшее,гражданский брак,953.0,5.771249
2,высшее,вдовец / вдова,135.0,5.185185
3,высшее,в разводе,315.0,4.761905
4,высшее,не женат / не замужем,803.0,5.728518
5,среднее,женат / замужем,8774.0,8.297242
6,среднее,гражданский брак,2964.0,10.290148
7,среднее,вдовец / вдова,776.0,6.958763
8,среднее,в разводе,826.0,7.627119
9,среднее,не женат / не замужем,1771.0,11.631846


Из таблицы видно, что уровень образования влияет значительнее, чем семейное положение. 
При этом внутри каждой категории образования сохраняется тренд, показывающий, что официально оформленные отношения позитивно влияют на вероятность выплатить кредит. 

In [62]:
#Зависимость от возраста и семейного положения.

age_marriage = pd.pivot_table(credit_history, index = ['family_status_id', 'age_group'], columns = 'debt', values = 'purpose', aggfunc = 'count')
age_marriage = age_marriage.reset_index()
age_marriage = age_marriage.merge(family_status_dict, on = 'family_status_id', how = 'right')

age_marriage['total_borrowers'] = age_marriage[0] + age_marriage[1]
age_marriage['debitors_percentage'] = age_marriage[1]/age_marriage['total_borrowers']*100
age_marriage = age_marriage.dropna()


age_marriage [['age_group', 'family_status','total_borrowers','debitors_percentage']].sort_values('age_group')


,age_group,family_status,total_borrowers,debitors_percentage
0,adult,женат / замужем,5006.0,7.471035
16,adult,не женат / не замужем,820.0,8.780488
4,adult,гражданский брак,1638.0,9.768010
8,adult,вдовец / вдова,189.0,7.407407
12,adult,в разводе,538.0,6.877323
1,junior,женат / замужем,428.0,9.579439
17,junior,не женат / не замужем,473.0,12.050740
5,junior,гражданский брак,304.0,9.539474
13,junior,в разводе,22.0,18.181818
14,middle,в разводе,209.0,9.090909


Видно, что более старшие группы клиентов лучше выплачивают кредит. При этом внутри каждой категории образования сохраняется тренд, показывающий, что официально оформленные отношения позитивно влияют на вероятность выплатить кредит.

In [63]:
#Зависимость от количества детей и уровня образования
children_and_education = pd.pivot_table(credit_history, index = ['education_id','children'], columns = 'debt', values = 'purpose', aggfunc = 'count')


children_and_education['total_borrowers'] = children_and_education[0] + children_and_education[1]
children_and_education['debitors_percentage'] = children_and_education[1]/children_and_education['total_borrowers']*100

children_and_education = children_and_education.dropna()
children_and_education = children_and_education.reset_index() 

children_and_education = children_and_education.merge(education_dict, on = 'education_id', how = 'right')

children_and_education[['children','education','total_borrowers', 'debitors_percentage']]


,children,education,total_borrowers,debitors_percentage
0,0.0,высшее,3259.0,5.308377
1,1.0,высшее,1330.0,5.413534
2,2.0,высшее,555.0,5.225225
3,3.0,высшее,83.0,4.819277
4,0.0,среднее,10152.0,8.254531
5,1.0,среднее,3288.0,10.431873
6,2.0,среднее,1400.0,10.714286
7,3.0,среднее,231.0,9.090909
8,4.0,среднее,32.0,12.500000
9,0.0,неоконченное высшее,486.0,7.613169


Из таблицы видно, что уровень образования влияет значительнее, чем количество детей. При этом, внутри каждой категории образования сохраняется тренд, показывающий, что увеличение количества детей негативно влияет на выплачиваемость кредита.

In [66]:
#Зависимость между возрастной группой и количеством детей. 

age_children = pd.pivot_table(credit_history, index = ['children', 'age_group'], columns = 'debt', values = 'purpose', aggfunc = 'count')

age_children = age_children.reset_index()
age_children = age_children.dropna()

age_children['total_borrowers'] = age_children[0] + age_children[1]
age_children['debitors_percentage'] = age_children[1]/age_children['total_borrowers']*100

age_children[['age_group', 'children', 'total_borrowers', 'debitors_percentage']].sort_values('age_group')







debt,age_group,children,total_borrowers,debitors_percentage
0,adult,0,4784.0,7.671405
4,adult,1,2257.0,8.684094
8,adult,2,964.0,8.195021
12,adult,3,163.0,8.588957
16,adult,4,18.0,5.555556
1,junior,0,891.0,10.662177
5,junior,1,279.0,10.752688
9,junior,2,56.0,10.714286
2,middle,0,2426.0,10.552350
6,middle,1,1750.0,10.742857


Из таблицы видно, что возрастная группа влияет значительнее, чем количество детей.
Внутри возрастных категорий нарушен тренд ухудшения платежеспособности при изменении количества детей. 
Делаем вывод, что количество детей незначительно влияет на выплачиваемость кредита. 

In [67]:
#Зависимость между количеством детей и семейным положением
children_marriage = pd.pivot_table(credit_history, index = ['family_status_id', 'children'], columns = 'debt', values = 'purpose', aggfunc = 'count')

children_marriage = children_marriage.reset_index()
children_marriage = children_marriage.merge(family_status_dict, on ='family_status_id', how = 'right')

children_marriage['total_borrowers'] = children_marriage[0] + children_marriage[1]
children_marriage['debitors_percentage'] = children_marriage[1]/children_marriage['total_borrowers']*100


children_marriage = children_marriage.dropna()

children_marriage[['family_status', 'children', 'total_borrowers', 'debitors_percentage']]





,family_status,children,total_borrowers,debitors_percentage
0,женат / замужем,0,7468.0,6.909480
1,женат / замужем,1,3004.0,8.222370
2,женат / замужем,2,1533.0,9.458578
3,женат / замужем,3,249.0,6.827309
4,женат / замужем,4,29.0,10.344828
6,гражданский брак,0,2729.0,8.391352
7,гражданский брак,1,1000.0,11.800000
8,гражданский брак,2,343.0,8.746356
9,гражданский брак,3,56.0,14.285714
12,вдовец / вдова,0,847.0,6.257379


In [69]:
#Зависимость от возраста и уровня образования

education_age = pd.pivot_table(credit_history, index =['age_group', 'education_id'], columns = 'debt', values = 'purpose', aggfunc='count')

education_age = education_age.reset_index()
education_age = education_age.dropna()

education_age['total_borrowers'] = education_age[0] + education_age[1]
education_age['debitors_percentage'] = education_age[1]/education_age['total_borrowers']*100


education_age = education_age.merge(education_dict, on = 'education_id', how = 'right')

education_age[['education', 'age_group', 'total_borrowers', 'debitors_percentage']]



,education,age_group,total_borrowers,debitors_percentage
0,высшее,adult,1959.0,4.594181
1,высшее,junior,318.0,6.918239
2,высшее,middle,1745.0,6.246418
3,высшее,retiree,1214.0,4.695222
4,среднее,adult,5933.0,9.000506
5,среднее,junior,732.0,13.387978
6,среднее,middle,3248.0,12.653941
7,среднее,retiree,5198.0,6.021547
8,неоконченное высшее,adult,204.0,9.803922
9,неоконченное высшее,junior,168.0,5.952381


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> Мне очень нравится, что ты сделал дополнительные проверки! Так работают настоящие аналитики

# Общие выводы:

1. Количество детей влияет на выплачиваемость кредита. Клиенты с 0 детьми выплачивают кредит лучше клиентов с большим количеством детей. Отклонения в пределах 2% 
2. Семейное положение влияет на выплачиваемость кредита в срок. Клиенты, официально оформившие отношения с партнером, выплачивают кредит лучше клиентов, не сделавших этого. 
3. Данные кореляции можно объяснить совокупностью факторов:
    1. Изменениями в образе жизни в связи с миграцией между категориями (расходы на детей или совместное планирование бюджета с партнером)
    2. Влиянием других категориальных значений на выплачиваемость кредита. 
4. Наиболее значительное влияние на вероятность выплаты кредита оказывает принадлежность клиента к возрастной группе, а также образованность. В рамках обеих категорий есть разброс до 10%. Лучше других выплачивают кредит люди, предпенсионной возрастной группы и люди, имеющие высшее образование.